# 7.c.ii Semantyka dystrybucyjna dokumentów

Musimy uaktualnić moduł gensim, po wykonaniu poniższej komórki proszę zresetować runtime

In [ ]:
!python3 -m pip install gensim==4.0.0b

Pobranie danych

In [ ]:
!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/doc2vec_data.zip?raw=true
!mv doc2vec_data.zip?raw=true doc2vec_data.zip
!unzip doc2vec_data.zip

### Tworzenie modelu doc2vec

#### Import potrzebnych pakietów

In [ ]:
from gensim.models import Word2Vec, Doc2Vec, KeyedVectors
from gensim.models.doc2vec import  TaggedDocument
import os

#### Wczytywanie danych i podział na paragrafy

In [ ]:
with open(os.path.join("doc2vec_data", "odyseja.txt")) as f:
    txt = f.read()
    odyseja = txt.split("\n\n")

with open(os.path.join("doc2vec_data", "iliada.txt")) as f:
    txt = f.read()
    iliada = txt.split("\n\n")


#### Definicja funkcji do preprocessingu
Funkcja w fazie treningowej zwraca obiekt TaggedDocument składający się z tokenów, oraz oznaczenia dokumentu (numeru na liście paragrafów), w fazie wykorzystania (inference stage) wystarczy sama lista Tokenów

In [ ]:
import re

tokenizer = re.compile(r"[\w]+")

def preprocess(doc, index=None):
    lowered = doc.lower()
    tokenized = tokenizer.findall(lowered)
    if index is not None:
        # training document indexed according to the corpus
        doc = TaggedDocument(tokenized, [index])
        return doc
    else:
        # inference stage
        return tokenized

#### Przetworzenie danych do treningu
Wypisanie liczby paragrafów w zbiorze treningowym

In [ ]:
total = iliada + odyseja
train_data = [preprocess(doc, i) for i, doc in enumerate(total)]
print(len(train_data))


#### Inicjalizacja i trening modelu
Parametr dm=0 oznacza że trenujemy model DBOW, który różni się od opisywanego w prezentacji modelu doc2vec tym, że nie zawiera już embeddingów dla pojedynczych słów.

In [ ]:
EPOCHS = 200
FEATURES_NUM = 500
model = Doc2Vec(vector_size=FEATURES_NUM, epochs=EPOCHS, dm=0)
model.build_vocab(train_data)
model.train(train_data, total_examples=model.corpus_count, epochs=EPOCHS)
dv = model.dv

#### Szukanie fragmentów tekstu
Stosujemy odległość kosinusową, by znaleźć paragraf najbardziej zbliżony do naszego opisu zdarzeń w tekście.

In [ ]:
#example = "Ojciec płaci okup za ciało Hektora."
#example = "Achilles płacze po stracie przyjaciela."
example = "Odys chroni się przed śpiewem syren."
preprocessed = preprocess(example)
example_vec = model.infer_vector(preprocessed)
for i, x in enumerate(dv.most_similar([example_vec], topn=5)):
    print(i+1, x[1], "\n", total[x[0]], "\n")

#### Zapisywanie modelu

In [ ]:
model.save("doc2vec.mdl")
model = Doc2Vec.load("doc2vec.mdl")

### Wykorzystanie word2vec do reprezentacji dokumentów

#### Trenowanie modelu

In [ ]:
import numpy

num_features = 100
train_sents = [d.words for d in train_data]

EPOCHS = 200
w2v_model = Word2Vec(vector_size=num_features, window=5, min_count=1)
w2v_model.build_vocab(train_sents)
w2v_model.train(train_sents, total_examples=model.corpus_count, epochs=EPOCHS)
w2v_wv = w2v_model.wv

#### Przygotowanie tablicy zanurzeń dla dokumentów
Definiujemy funkcję agregującą wektory dla paragrafu, jako średni wektor tokenów.
Przygotowujemy obiekt, przechowujący wektory, dla którego kluczem są numery paragrafów, a wartościami uśrednione wektory paragrafów.

In [ ]:
def w2v_for_docs(doc):
    tokenized = preprocess(doc)
    unk = numpy.zeros((num_features,))
    vex = []
    for tok in tokenized:
        try:
            vex.append(w2v_wv[tok])
        except KeyError:
            vex.append(unk)
    avg = numpy.average(vex, axis=0)
    return avg

vec_table = [w2v_for_docs(doc) for doc in total]
kv = KeyedVectors(num_features)
indices = list(range(len(vec_table)))
kv.add_vectors(indices, vec_table)

#### Szukanie fragmentów tekstu

In [ ]:
#example = "Ojciec płaci okup za ciało Hektora."
#example = "Achilles płacze po stracie przyjaciela."
example = "Odys chroni się przed śpiewem syren"

example_vec = w2v_for_docs(example)
for i, x in enumerate(kv.most_similar([example_vec], topn=5)):
    print(i+1, x[1], "\n", total[x[0]], "\n")